# インポート

In [ ]:
#ほぼ絶対使用
import pandas as pd
import numpy as np
import scipy.stats as st
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
import itertools
from hyperopt import fmin,tpe,hp,STATUS_OK,Trials
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,auc,log_loss,roc_curve,roc_auc_score

#場合により使用
from sklearn.svm import LinearSVC
import cv2
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from scipy.sparse import csr_matrix
import MeCab
tagger = MeCab.Tagger()
mecab = MeCab.Tagger('-Owakati')
import re
from collections import Counter
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,auc,log_loss,roc_curve,roc_auc_score
import csv
import codecs

# ファイルのインポートと説明変数の設定

In [ ]:
train=pd.read_csv('..//Downloads//1.train_data.csv'encoding=ANSI)

tr_x=train2.drop( 'カラム名',axis=1)
tr_y=train2['応答変数']

# 前処理

In [ ]:
#数値データとカテゴリデータをわける
num_features = [数値データ]

cat_features = [カテゴリデータ]

features = num_features + cat_features

In [ ]:
#カテゴリ変数を数値に変換
le = LabelEncoder()
le_dict = {}
for col in cat_features:
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])
    le_dict['col']=le

#タイプをcategory型に変換する
train[cat_features] = train[cat_features].astype('int')
test[cat_features] = test[cat_features].astype('int')

# 欠損値の確認

In [ ]:
print(train.isnull().any())

# 基本情報の確認

In [ ]:
test.info()

In [ ]:
train.describe()

# 相互特徴量の作成

In [ ]:
for c in num_features:
    for v in num_features:
        if c==v:
            continue
        combine[c+"+"+v]=combine[c]+combine[v]
        combine[c+"-"+v]=combine[c]-combine[v]
        combine[c+"-"+v]=combine[c]-combine[v]
        combine[c+"-"+v]=abs(combine[c+"-"+v])
        if 0 in combine[c] or 0 in combine[v]== True:
            combine[c+"*"+v]=0
        combine[c+"/"+v]=combine[c]/combine[v]
        if 0 in combine[c] or 0 in combine[v]== True:
            combine[c+"/"+v]=0
            
for c in combine.columns:
    con = collections.Counter(combine[c]==0)
    if con[1] == combine.shape[0]:
        combine=combine.drop(c,axis=1)

# NLP

In [ ]:
def extractKeyword(text):  
      #\\\textを形態素解析して、名詞のみのリストを返す\\\  
    tagger = MeCab.Tagger()  
    encoded_text=text  
    #encoded_text = text.encode('utf-8')  
    #print(encoded_text)  
    #encoded_text = encoded_text.replace('\ '  ' ')  
    node = tagger.parseToNode(encoded_text).next  
    keywords = []
    while node:  
        if node.feature.split(' ')[0] == "名詞":  
            keywords.append(node.surface)  
        node = node.next  
    return keywords  

def splitDocument(documents):  
  
      #\\\文章集合を受け取り、名詞のみ空白区切りの文章にして返す\\\  
    splitted_documents = []  
  
    for d in documents:  
        keywords = extractKeyword(d)  
        splitted_documents.append(' '.join(keywords))  
    return splitted_documents  
      
      
def removeStoplist(documents,stoplist):  
      #\\\ストップワードを取り除く\\\\  
    stoplist_removed_documents = []  
    for document in documents:  
        words = []  
        if isinstance(document,int) == True:  
            document=str(document)  
        for word in document.lower().split():  
            if word not in stoplist:  
                words.append(word)  
        stoplist_removed_documents.append(words)  
    return stoplist_removed_documents  
      
      
def getTokensOnce(all_tokens):  
  
      
    #  u\\\一回しか出現しない単語を返す\\\  
    tokens_once = set([])  
      
    for word in set(all_tokens):  
        if all_tokens.count(word) == 1:  
            tokens_once.add(word)  
    return tokens_once  
       
def removeTokensOnce(documents,tokens_once):  
    #  u\\\一回しか出現しない単語を取り除く\\\  
    token_removed_documents = []  
    for document in documents:  
        words = []  
        for word in document:  
            if word not in tokens_once:  
                words.append(word)  
        token_removed_documents.append(words)  
    return token_removed_documents 

In [ ]:
if __name__ == \__main__\:  
    raw_documents =df_train['text']  
  
      # 空白区切りの文字列を入れる  
    splitted_documents = splitDocument(raw_documents)  
      
    print(\splitted_documents:\)  
    #for d in splitted_documents:  
        #print(d)  
        #print(\\)  
      
    stoplist = set([  
    #    'の'  'に'  'て'  '、'  'が'  'た'  '。'  'は'  'を'  'で' '.' '\ ' '/' 't' 'https' '://' 'co'  
    ]) # 通常の文字列でよい  
    print(\stoplist:\)  
    #for s in stoplist:  
        #print(s)  
      
      # ストップワードを除いた二重リスト  
    stoplist_removed_documents = removeStoplist(splitted_documents  stoplist)  
      
    print(\stoplist_removed_documents:\)  
    #for t in stoplist_removed_documents:  
        #for w in t:  
            #print(w)   
            #print(\\)  
        #print(\\)  
      
      # 全ての単語を重複ありで含むリスト  
    all_tokens = ' '.join(stoplist_removed_documents)  
    print(\all_tokens:\)  
    #for t in all_tokens:  
        #print(t)   
        #print(\\)  
      
      # 一回しかでてこない単語のみを持つセット  
    tokens_once = getTokensOnce(all_tokens)  
    print(\tokens_once:\)  
    #for t in tokens_once:  
        #print(t)   
        #print(\\)  
      # 一回しかでてこない単語を除いた最終的なテキスト  
    preprocessed_documents = removeTokensOnce(stoplist_removed_documents  tokens_once)  
    print(\preprocessed_documents:\)  
    #for d in preprocessed_documents:  
        #for w in d:  
            #print(w)   
        #print(\\)


In [ ]:
# この関数を使って，textを分かち書き．
# applyをすることでtextの1行1行に対して分かち書き．

mecab = MeCab.Tagger('-Owakati')
df_train['text'] = df_train['text'].apply(mecab.parse)
df_train.head()


# targetが「0」つまり，災害に関係ないツイートの上位20件の頻出単語です
df_train.groupby('flg')['text'].apply(lambda x: Counter(' '.join(x).split(' ')).most_common(n=20)).iloc[0]


# targetが「0」の場合の上位10件の頻出単語をストップワードとして指定
stop_words = set([
    'の', 'に', 'て', '、', 'が', 'た', '。', 'は', 'を', 'で'
])
df_train.groupby('flg')['text'].apply(lambda x: Counter([w for w in ' '.join(x).split(' ') if w not in stop_words]).most_common(n=20)).iloc[0]



# テストデータの分割

In [ ]:
# トレーニング用データをトレーニング用/バリデーション用に分割，バリデーションは2割分
train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=0)

# indexをリセット
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
train_df.head()

# クロスバリデーション

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

scores=[]

# KFoldクラスを用いてクロスバリデーションの分割を行う
kf=KFold(n_splits=4,shuffle=True,random_state=71)
for tri_x,vai_x in kf.split(train_x):
    tr_x,va_x = train_x.iloc[tri_x],train_x.iloc[vai_x]
    tr_y,va_y = train_y.iloc[tri_x],train_y.iloc[vai_x]
    
    #モデルを用いて予測（サンプル）
    model=Model()
    model.fit(tr_x,tr_y,va_x,va_y)
    va_pred=model.predict(va_x)
    score=log_loss(va_y,va_pred)
    scores.appemd(score)
    
#各スコアの平均をとる
print(np.mean(scores))
print(scores)

# Beg Of Words

In [ ]:
# CountVectorizerを定義します
cv = CountVectorizer()
tf = TfIdfVectorizer()

# CountVectorizerに訓練用のtextを学習させます
cv.fit(train_df['text'].values)
cv.get_feature_names()[:10]

# トレーニング用データにCountVectorizerを適用
X_train = cv.transform(train_df['text'].values)
y_train = train_df['flg'].values

# バリデーション用データにCountVectorizerを適用
X_val = cv.transform(val_df['text'].values)
y_val = val_df['flg'].values

# モデルいろいろ

# ハイパーパラメータ調整

In [ ]:
#モデル作成
params=lgb_params()
#探索回数
max_evals=10
trials=Trials()
history=[]
#パラメータを変える
fmin(lgbscore,params,algo=tpe.suggest,trials=trials,max_evals=max_evals)

history=sorted(history,key=lambda tpl:tpl[1])
best=history[0]
print(f'best params:{best[0]},score:{best[1]:4f}')

# 作り方

params→model→scoreの順

In [ ]:
#モデル作成
params=lgb_params()
max_evals=10
trials=Trials()
history=[]
fmin(lgbscore,params,algo=tpe.suggest,trials=trials,max_evals=max_evals)

history=sorted(history,key=lambda tpl:tpl[1])
best=history[0]
print(f'best params:{best[0]},score:{best[1]:4f}')

# サポートベクタマシン（分類）

In [ ]:
# LinearSVCを定義します(random_state=0でseed値を固定)
l_svc = LinearSVC(random_state=0)

# fitで学習します
l_svc.fit(X_train, y_train)

# サポートベクタマシン（回帰）

In [ ]:
#Opencv p196
def train_svm(tr_x,tr_y):
    svm=cv2.ml.SVM_create()
    svm.train(tr_x,cv2.ml.ROW_SAMPLE,y_train)
    return svm

# XGBoost

In [ ]:
def xgb_Model(params):
    params['max_depth']=int(params['max_depth'])
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_val, label=y_val)
    num_round=10000
    #watchlist=[(dtrain,'train'),(dtest,'eval')]
    model=xgb.train(params,dtrain,num_round)
    va_pred=model.predict(dtest)
    va_pred=np.where(va_pred>0.5,1,0)
    return va_pred

In [ ]:
def xgbscore(params):
    xgb_model=xgb_Model(params)
    va_pred=xgb_model.predict(X_val2)
    va_pred=np.where(va_pred>0.5,1,0)
    score=f1_score(y_val,va_pred)
    history.append((params,score))
    return {'loss': score,'status':STATUS_OK}

In [ ]:
def xgb_params():
    params={
        'booster':'gbtree',
        'objective':'binary:logistic',
        'eta':0.1,
        'gamma':hp.loguniform('gamma',np.log(1e-8),np.log(1.0)),
        'max_depth':hp.quniform('max_depth',3,9,1),
        'min_child_weight':hp.loguniform('min_child_weight',np.log(0.1),np.log(10)),
        'colsample_bytree':hp.quniform('colsample',0.6,0.95,0.05),
        'subsample':hp.quniform('subsample',0.6,0.95,0.05),
        'eval_metric': 'error',
        'early_stopping_rounds': 100,
        'random_state':71,
    }
    return params

# LightGBM

In [ ]:
#基準のparam
params={
        'objective':'binary',
        'metric':'binary_error',
        'learning_rate':0.1,
        'max_depth':5,
        'min_data_in_leaf':5,
        'seed':71,
        'feature_fraction':0.8,
}

In [ ]:
def lgb_params():
    params={
        'objective':'binary',
        'metric':'binary_error',
        'learning_rate':0.1,
        'max_depth':hp.quniform('max_depth',3,9,5),
        'min_data_in_leaf':hp.quniform('min_data_in_leaf',3,5,9),
        'seed':71,
        'feature_fraction':0.8,
        'bagging_fraction':hp.quniform('bagging_fraction',0.6,0.8,0.2),
    }
    return params

In [ ]:
def lgbscore(params):
    lgb_model=lgb_Model(params)
    va_pred=lgb_model.predict(X_val)
    va_pred=np.where(va_pred>0.5,1,0)
    score=f1_score(y_val,va_pred)
    history.append((params,score))
    return {'loss': score,'status':STATUS_OK}

In [ ]:
def lgb_Model(params):
    params['max_depth']=int(params['max_depth'])
    params['min_data_in_leaf']=int(params['min_data_in_leaf'])
    lgb_train=lgb.Dataset(X_train,y_train)
    lgb_eval=lgb.Dataset(X_val,y_val)
    #categorical_features = ['predict']
    
    lgbmodel = lgb.train(params,
                lgb_train,
                valid_sets=(lgb_train, lgb_eval),
                num_boost_round=10000,
                early_stopping_rounds=100,
                verbose_eval=50)
    return lgbmodel

# 線形モデル

In [ ]:
from sklearn.linear_model import LogisticRegression
#回帰の場合はRidge mse

scaler=StandardScaler()
tr_x=scaler.fit_transform(tr_x)
va_x=scaler.fit_transform(va_x)
test_x=scaler.fit_transform(test_x)

model=LogisticRegression(C=1.0)
model.fit(tr_x,tr_y)

#バリデーションデータでのスコアの確認
#確率を出力
va_pred=model.predict_proba(va_x)
score=log_loss(va_y,va_pred)
print(f'log_loss:{score:4f}')

#予測
pred=model.predict(test_x)

# ニューラルネットワーク

In [ ]:
from keras.layers import Dense,Dropout
from keras.models import Sequential
from sklearn.preprocessing impor StandardScaler

#データのスケーリング
scaler=StandardScaler()
tr_x=scaler.fit_transform(tr_x)
va_x=scaler.fit_transform(va_x)
test_x=scaler.fit_transform(test_x)
#モデルの構築
model=Sequential()
model.add(Dense(256,activation='relu',input_shape=(train_x.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

#学習の実行
#バリデーションデータもモデルに渡し、学習の振興とともにスコアがどう変わるかモニタリングする
batch_size=120
epochs=10
history=model.fit(tr_x,tr_y,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(va_x,va_y))

#スコアの確認
va_pred=model.predict(va_x)
score=log_loss(va_y,va_pred,eps=1e-7)
print(f'log_loss:{score:4f}')

#予測
pred=model.predict(test_x)

# コールバック

In [ ]:
学習時、ミニバッチの処理やエポックごとに指定した処理を走らせることができる

アーリーストッピング
モデルの保存
学習のスケジューリング
ログ、可視化

In [ ]:
from keras.callbacks import EarlyStopping
#アーリーストッピング(20)
epochs=50
early_stopping=EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True)

history=model.fit(tr_x,tr_y,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(va_x,va_y),callbacks=[early_stopping])
pred=model.predict(test_x)

# パラメータ

In [ ]:
from hyperopt import hp
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import ReLU,PReLU
from keras.layers.core import Dense,Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.optimizers import SGD,Adam
from sklearn.preprocessing import StandardScaler

In [ ]:
def score(params):
    model-MLP(params)
    model.fit(tr_x,tr_y,va_x,va_y)
    va_pred=model.predict(va_x)
    score=log_loss(va_y,va_pred)
    print(f'log_loss:{score:4f}')
    
    #情報を記録しておく
    history.append((params,score))
    
    return {'loss':score,'status':STATUS_OK}

In [ ]:
base_param={
    'input_dropout':0.0,
    'hidden_layers':3,
    'hidden_units':96,
    'hidden_acivation':'relu',
    'hidden_dropout':0.2,
    'batch_norm':'before_act',
    'optimizer':{'tyoe':'adam','lr':0.001},
    'beach_size':64
}

param_space={
    'input_dropout':hp.quniform('input_dropout',0,0.2,0.05),
    'hidden_layers':hp.quniform('hidden_layers',2,4,1),
    'hidden_units':hp.quniform('hidden_units',32,256,32),
    'hidden_acivation':hp.choice('hidden_activation',['prelu','relu']),
    'hidden_dropout':hp.quniform('hidden_dropout',0,0.3,0.05),
    'batch_norm':hp.choice('batch_norm',['before_act','no']),
    'optimizer':hp.choice('optimizer',[{'type':'adam','lr';hp.loguniform('adam_lr',np.log(0.00001),np.log(0.01))}]),
    'beach_size':hp.quniform('batch_size',32,128,32)
}

In [ ]:
class MLP:
    def __init__(self,params):
        self.params=params
        self.scaler=None
        self.model=None
        
    def fit(self,tr_x,tr_y,va_x,va_y):
        input_dropout=self.params['input_dropout']
        hidden_layers=int(self.params['hidden_layers'])
        hidden_units=int(self.params['hidden_units'])
        hidden_activation=self.params['hidden_activation']
        hidden_dropout=self.params['hidden_dropout']
        betch_norm=self.params['beach_norm']
        optimizer_tyoe=self.params['optimizer']['tyoe']
        optimizer_lr=self.params['optimizer']['lr']
        batch_size=int(self.params['batch_size'])
        
        #標準化
        self.scaler=StandardScaler()
        tr_x=self.scaler.fit_transform(tr_x)
        va_x=self.scaler.fit_transform(va_x)
        
        self.model=Sequential()
        
        #入力層
        self.model.add(Dropout(input_dropout,input_shape=(tr_x.shape[1],)))
        
        #中間層
        for i in range(hidden_layers):
            self.model.add(Dense(hidden_units))
            if batch_norm=='before_act':
                self.model.add(BatchNormalization())
            if hidden_activation=='prelu':
                self.model.add(PReLU())
            elif hidden_activation='relu':
                self.model.add(Relu())
            else:
                raise NotImplementedError
            self.model.add(Dropout(hidden_dropout))
        
        #出力層
        self.model.add(Dencse(1,activation='sigmoid'))
        
        #オプティマイザ
        if optimizer_type='sgd':
            optimizer=SGD(lr=optimizer_lr,decay=1e-6,momentum=0.9,nesterov=True)
        elif optimizer_type='adam':
            optimizer=ADAM(lr=optimizer_lr,beta_1=0.9,beta_2=0.999,decay=0)
        else:
            raise NotImplementedError
            
        #目的変数、評価指標などの設定
        self.model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
        
        epochs=200
        patience=20
        early_stopping=EarlyStopping(patience=patience,restore_best_weights=True)
        
        #学習の実行
        
        history=self.model.fit(tr_x,tr_y,epochs=epochs,batch_size=batch_size,verbose=1,validation_data=(va_x,va_y),callbacks=early_stopping)
        
        
        
    def predict(self,x):
        x=self.scaler.transfform(x)
        y_pred=self.model.predict(x)
        y_pred=y_pred.flatten()
        return y_pred

# 精度評価

# 混同行列

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print('Normalized confusion matrix')
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment='center',
                 color='white' if cm[i, j] > thresh else 'black')

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_val, l_svc.predict(X_val))
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure(figsize=(5, 5))
plot_confusion_matrix(cnf_matrix, classes=[i for i in range(2)],
                      title='Confusion matrix, without normalization')

plt.show()

# 提出手順

In [ ]:
# 提出用のコード　パラメータはモデルに合わせて設定する
mecab = MeCab.Tagger('-Owakati')
df_test['text'] = df_test['text'].apply(mecab.parse)
X_test = cv.transform(df_test['text'].values)
X_test = X_test.astype(float)
params={
        'objective':'binary',
        'metric':'binary_error',
        'learning_rate':0.1,
        'max_depth':5,
        'min_data_in_leaf':5,
        'seed':71,
        'feature_fraction':0.8,
}
model=lgb_Model(params)


# 予測しています
predict =model.predict(X_test)
predict=np.where(predict>0.5,1,0)



In [ ]:
# カラムのみのcsvファイルsubmission.csvを用意する
submission = pd.read_csv('./data/3.submission.csv')

#予想結果をカラムへ代入
submission['flg'] = predict

#ファイルへ書き出し、保存
submission.to_csv('submission.csv', index=False)

# 相関係数
相関係数が大きければその特徴量を選択

In [ ]:
corrs=[]
for c in tr_x.columns:
    corr=np.corrcoef(tr_x[c],tr_y)[0,1]
    corrs.append(corr)
corrs=np.array(corrs)

corrsp=[]
for c in tr_x.columns:
    corrp=st.spearmanr(train4[c],tr_y).correlation
    corrsp.append(corrp)
corrsp=np.array(corrsp)
    
    
idx=np.argsort(np.abs(corrs))[::-1]
topcols,topimp=train4.columns.values[idx][:5],corrs[idx][:5]
print(topcols,topimp)

    
idx2=np.argsort(np.abs(corrsp))[::-1]
topcols2,topimp2=train4.columns.values[idx2][:5],corrs[idx2][:5]
print(topcols2,topimp2)

# カイ二乗統計量による特徴量選択

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler as MM
import numpy as np
from sklearn.feature_selection import mutual_info_regression

#カイ二条統計量
x=MM().fit_trainsform(train_x)
c2,_=chi2(x,train_y)

#重要度の上位を出力

idx=np.argsort(c2)[::-1]
topcols,topimp=train_x.colmuns.values[idx][:5],corrs[idx][:5]
print(topcols,topimp)

# 相互情報量による特徴量選択

In [ ]:
from sklearn.feature_selection import mutual_info_classif as mc

#相互特徴量
mi=mc(train_x,train_y)

#重要度の順位を出力する

idx=np.argsort(c2)[::-1]
topcols,topimp=train_x.colmuns.values[idx][:5],corrs[idx][:5]
print(topcols,topimp)

# threshold_search

In [ ]:
from sklearn.metrics import f1_score
from tqdm import tqdm


def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in tqdm([i * 0.01 for i in range(100)], disable=True):
        score = f1_score(y_true=y_true, y_pred=y_proba > threshold)
        if score > best_score:
            best_threshold = threshold
            best_score = score
    search_result = {'threshold': best_threshold, 'f1': best_score}
    return search_result